In [4]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
from matplotlib.backends.backend_agg import FigureCanvasAgg as FigureCanvas
from matplotlib.figure import Figure

In [5]:
def siftTrack(trainImg,trainKP,trainDesc,QueryImgBGR,h,w):
    MIN_MATCH_COUNT=50

    detector=cv2.xfeatures2d.SIFT_create()

    FLANN_INDEX_KDITREE=0
    flannParam=dict(algorithm=FLANN_INDEX_KDITREE,tree=5)
    searchParam = dict(checks=50)
    #flann=cv2.FlannBasedMatcher(flannParam,searchParam)
    flann=cv2.FlannBasedMatcher(flannParam,{})

    #trainImg=cv2.imread("demo0.png",0)
    #trainKP,trainDesc=detector.detectAndCompute(trainImg,None)

    #cam=cv2.VideoCapture(0)
    #while True:
    #ret, QueryImgBGR=cam.read()
    
    QueryImg=cv2.cvtColor(QueryImgBGR,cv2.COLOR_BGR2GRAY)
    queryKP,queryDesc=detector.detectAndCompute(QueryImg,None)
    matches=flann.knnMatch(queryDesc,trainDesc,k=2)

    goodMatch=[]
    queryBorder = None
    qp = None
    for m,n in matches:
        if(m.distance < 0.75*n.distance):
            goodMatch.append(m)
    if(len(goodMatch) > MIN_MATCH_COUNT):
        tp=[]
        qp=[]
        for m in goodMatch:
            tp.append(trainKP[m.trainIdx].pt)
            qp.append(queryKP[m.queryIdx].pt)
        tp,qp=np.float32((tp,qp))
        H,status=cv2.findHomography(tp,qp,cv2.RANSAC,3.0)
        
        #h,w=trainImg.shape
        trainBorder=np.float32([[[0,0],[0,h-1],[w-1,h-1],[w-1,0]]])
        queryBorder=cv2.perspectiveTransform(trainBorder,H)
        cv2.polylines(QueryImgBGR,[np.int32(queryBorder)],True,(0,255,0),5)
        return (1,queryBorder,qp)
        #print(queryBorder)
    else:
        #print("Not Enough match found")
        return (-1,queryBorder,qp)



In [1]:
def findOrientation(qp):
    tl = qp[0][0] #top left
    bl = qp[0][1] #botton left
    br = qp[0][2] #botton right
    tr = qp[0][3] #top right
    center = [(t1[0]+br[0])/2,(t1[1]+br[1])/2]
    return center
    

In [ ]:
cam = cv2.VideoCapture(0)
img1 = cv2.imread("demo.jpg")
#img3 = cv2.imread("demo0.png")
h,w,d = img1.shape
sift = cv2.xfeatures2d.SIFT_create()
kp1, des1 = sift.detectAndCompute(img1,None)
while True:
    ret, img2=cam.read()
    checker,dst,qp = siftTrack(img1,kp1,des1,img2,h,w)
    mask = np.zeros_like(img2)
    if checker != -1:
        center = findOrientation(qp)
        res = cv2.resize(img, None, fx=2, fy=2, interpolation = cv.INTER_AREA)
        print("YES")
    else:
        print("NO")
    keypressed = cv2.waitKey(5)
    if keypressed == 27:
        break
    cv2.imshow('mask',img2)
    
cam.release()
cv2.destroyAllWindows()

NO
NO
NO
[ 887.79364014  331.79214478]
[ 908.36956787  744.36608887]
[ 1196.58215332   726.0848999 ]
[ 1158.37854004   302.68963623]
YES
[ 954.41827393  325.73849487]
[ 971.36224365  744.77478027]
[ 1266.0345459    734.93908691]
[ 1231.91040039   303.93533325]
YES
[ 934.41021729  322.9538269 ]
[ 953.39141846  746.12512207]
[ 1249.02380371   732.87054443]
[ 1212.69873047   298.9881897 ]
YES
[ 931.05932617  327.35232544]
[ 952.4541626   752.71783447]
[ 1247.7434082    738.87359619]
[ 1210.29162598   301.80886841]
YES
[ 913.61437988  321.53536987]
[ 936.14923096  744.74279785]
[ 1231.2479248    728.76550293]
[ 1192.0078125    294.93096924]
YES
[ 898.5916748   345.98937988]
[ 926.0234375   774.42272949]
[ 1221.55627441   754.79656982]
[ 1176.66552734   315.49102783]
YES
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
NO
